In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
import numpy as np
import warnings
import os

In [2]:
df = pd.read_csv("Consumer_Complaints (1).csv")
df.head()

C:\Users\Devyani\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,07/17/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,NaN,NaN,ALLY FINANCIAL INC.,TX,75035,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309495
1,07/17/2019,Credit card or prepaid card,General-purpose prepaid card,Trouble using the card,Problem using the card to withdraw money from ...,NaN,NaN,SQUARE INC,KS,674XX,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3310031
2,07/17/2019,Debt collection,Other debt,Took or threatened to take negative or legal a...,Threatened or suggested your credit would be d...,NaN,NaN,"Diversified Consultants, Inc.",FL,NaN,NaN,NaN,Web,07/17/2019,In progress,Yes,NaN,3309687
3,07/17/2019,Mortgage,VA mortgage,Trouble during payment process,NaN,NaN,NaN,"FLAGSTAR BANK, FSB",VA,22554,Servicemember,NaN,Web,07/17/2019,In progress,Yes,NaN,3308925
4,07/17/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes it acted appropriately as aut...,BYL Collection Services,TN,370XX,Servicemember,NaN,Web,07/17/2019,Closed with explanation,Yes,NaN,3308914


In [3]:
df = df.dropna()
df.isnull().sum()

Date received                   0
Product                         0
Sub-product                     0
Issue                           0
Sub-issue                       0
Consumer complaint narrative    0
Company public response         0
Company                         0
State                           0
ZIP code                        0
Tags                            0
Consumer consent provided?      0
Submitted via                   0
Date sent to company            0
Company response to consumer    0
Timely response?                0
Consumer disputed?              0
Complaint ID                    0
dtype: int64

In [4]:
df = df.iloc[:100] 
df.shape

(100, 18)

In [5]:
corpus = df["Consumer complaint narrative"].tolist()

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
from nltk.corpus import stopwords
from string import punctuation
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Devyani\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
final_corpus= []
for i in range(len(corpus)):
    text = word_tokenize(corpus[i].lower())
    text = [t for t in text if len(t) > 2]
    text = [lemmatizer.lemmatize(y) for y in text if y not in stuff_to_be_removed]
    sent = " ".join(text)
    final_corpus.append(sent)

In [8]:
new_df = pd.DataFrame(final_corpus)
new_df.columns = ["Complaint"]
new_df.head()

,Complaint
0,charge made company making false claim product...
1,debt beyond maryland statute limitation illega...
2,personal belonging vehicle stolen xxxx =my loc...
3,complaint necessary ensure recovery collection...
4,contacted judgement xxxx whic javitch block go...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector= tfidf.fit_transform(new_df["Complaint"])

In [10]:
X = vector.toarray()

In [11]:
lsa = TruncatedSVD(n_components=3, n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=3, n_iter=100,
       random_state=None, tol=0.0)

In [12]:
words = tfidf.get_feature_names()

In [14]:
for i,y in enumerate(lsa.components_):
    componentwords = zip(words, y)
    sortedComponentwords = sorted(componentwords, key = lambda x: x[1],
                                reverse=True)
    sortedComponentwords = sortedComponentwords[:10]
    print ("\nTopic", i, ";")
    for x in sortedComponentwords:
        print (x)


Topic 0 ;
('xxxx', 0.5973886296156211)
('xx', 0.40929085547564675)
('debt', 0.2199536024283688)
('account', 0.17457450734934477)
('credit', 0.15687916712680253)
('company', 0.1501008624178641)
('collection', 0.1351216028655499)
('report', 0.1194867619121847)
('never', 0.09783851335915722)
('00', 0.09735665986715228)

Topic 1 ;
('xx', 0.43574552576390424)
('xxxx', 0.2650888864979552)
('late', 0.06710615195889881)
('bill', 0.053981295290400653)
('call', 0.05376370105215103)
('equipment', 0.0404179680251791)
('hospital', 0.038252956757268995)
('veteran', 0.03800324151522179)
('uheaa', 0.037277339369244857)
('opened', 0.03552232419116649)

Topic 2 ;
('xx', 0.4210731950971783)
('report', 0.1892375910443573)
('account', 0.1750616607571788)
('company', 0.164238250672672)
('credit', 0.15820547123630246)
('give', 0.12285018744875088)
('collectable', 0.10701878447504806)
('effort', 0.10701878447504806)
('date', 0.10372747374885656)
('showing', 0.09891783554518911)
